# Gradient Descent and Loss Functions

Authors: [Alexander Wang](https://github.com/awang9999), [Winnie Chow](https://github.com/winnie0617)

This module introduces the gradient descent algorithm for optimization and loss functions within the context of machine learning. This material is intended to be delivered on April 16, 2022 at a Cornell Data Science onboarding presentation, but is sufficient as standalone material as well.

Topics covered:
* Loss functions (MSE, MAE, Binary Cross Entropy, Hinge Loss)
* Gradient Descent (general algorithm and intuition, proof of convergence omitted)
* Gradient Descent variants (SGD, Mini-batched SGD)
* Brief discussion on Newton's method
* Example of applying gradient descent to solve the optimization problem for multidimensional linear regression (using MSE loss).

Associated slides for this presentation can be found [here](https://docs.google.com/presentation/d/1Er7sTuaL892dptan2unGSKmaC3BG7pKy8oNUIteFOhA/edit#slide=id.p).

### Part 1: Motivation

Up until now “training” models have largely been a black box. We talk about it all the time, but how do we REALLY adjust the parameters of a model? What is a good measure for how well a model predicts the training data? How do we adjust the model parameters based on the model’s performance in predicting the training data automatically?

This module aims to answer these questions.

### Part 2: Consider this problem



In [1]:
import sys
print(sys.version)
# Require versions 3.7, 3.8 or 3.9

3.9.5 (default, Jan  2 2022, 22:23:27) 
[GCC 11.1.0]


In [3]:
import numpy as np